In [1]:
# #Resize based off of smallest
# from PIL import Image
# import os

# def find_smallest_image_size(directories):
#     min_width, min_height = float('inf'), float('inf')
#     for directory in directories:
#         for filename in os.listdir(directory):
#             if filename.endswith('.jpg'):  
#                 filepath = os.path.join(directory, filename)
#                 with Image.open(filepath) as img:
#                     width, height = img.size
#                     min_width, min_height = min(min_width, width), min(min_height, height)
#     return min_width, min_height

# def resize_images_to_smallest(src_directory, dest_directory, min_width, min_height):
#     os.makedirs(dest_directory, exist_ok=True)  # Ensure destination directory exists
#     for filename in os.listdir(src_directory):
#         if filename.endswith('.jpg'):  
#             filepath = os.path.join(src_directory, filename)
#             with Image.open(filepath) as img:
#                 resized_img = img.resize((min_width, min_height), Image.Resampling.LANCZOS)
#                 resized_img.save(os.path.join(dest_directory, filename))

# dir1_src = '/home/cygnus/workspace/dl/project/hawks'
# dir2_src = '/home/cygnus/workspace/dl/project/non_hawks'  
# dir1_dest = '/home/cygnus/workspace/dl/project/hawks_resize'  
# dir2_dest = '/home/cygnus/workspace/dl/project/non_hawks_resize'  

# # Find the smallest size across both directories
# min_width, min_height = find_smallest_image_size([dir1_src, dir2_src])
# print(f"Smallest size found: {min_width}x{min_height}")

# # Resize images in both directories
# resize_images_to_smallest(dir1_src, dir1_dest, min_width, min_height)
# resize_images_to_smallest(dir2_src, dir2_dest, min_width, min_height)

Smallest size found: 304x321


In [2]:
import numpy as np
import cv2
import pandas as pd
import os

# Define the path to the directory containing your image files
image_dir = '/workspace/project/master_img/'

# Load CSV file containing image file names and labels
csv_file = '/workspace/project/master.csv'
df = pd.read_csv(csv_file, header=None, names=['image_file_name', 'label'], skiprows=1)

# Construct the full file paths for the images
df['full_image_path'] = df['image_file_name'].apply(lambda x: os.path.join(image_dir, x))

# Extract full image file paths and labels from the DataFrame
image_file_paths = df['full_image_path'].tolist()
labels = df['label'].tolist()

# Determine the shape of your images
image_shape = (321, 304, 3) # Replace with actual shape

# Initialize parameters for memory-mapped array
im_memmap_path = 'images.memmap'
labels_memmap_path = 'labels.memmap'

# Create memory-mapped array files
images_memmap = np.memmap(im_memmap_path, dtype=np.float32, mode='w+', shape=(len(image_file_paths), *image_shape))
labels_memmap = np.memmap(labels_memmap_path, dtype=bool, mode='w+', shape=(len(labels)))

# Load images and labels into memory-mapped arrays
for idx, (image_file_path, label) in enumerate(zip(image_file_paths, labels)):
    image = cv2.imread(image_file_path)
    if image is not None:
        image = image.astype(np.float32) / 255.0
        images_memmap[idx] = image
        labels_memmap[idx] = label
    else:
        print(f"Failed to load image at {image_file_path}")

# Shuffle indices
random_seed = 47
np.random.seed(random_seed)
indices = np.arange(len(images_memmap))
np.random.shuffle(indices)

# Create shuffled memory-mapped arrays
shuffled_im_memmap_path = 'shuffled_images.memmap'
shuffled_labels_memmap_path = 'shuffled_labels.memmap'

shuffled_images = np.memmap(shuffled_im_memmap_path, dtype=np.float32, mode='w+', shape=images_memmap.shape)
shuffled_labels = np.memmap(shuffled_labels_memmap_path, dtype=bool, mode='w+', shape=labels_memmap.shape)

# Copy shuffled data to new memory-mapped arrays
for i, idx in enumerate(indices):
    shuffled_images[i] = images_memmap[idx]
    shuffled_labels[i] = labels_memmap[idx]

print("Shuffled images shape:", shuffled_images.shape)
print("Shuffled labels shape:", shuffled_labels.shape)



Shuffled images shape: (6396, 321, 304, 3)
Shuffled labels shape: (6396,)


In [1]:
# #Putting the Images into  numpyarrays
# import numpy as np
# import cv2
# import pandas as pd
# import os
# import PIL

# # Define the path to the directory containing your image files
# #image_dir = '/home/cygnus/workspace/dl/project/master_img/'
# image_dir = '/workspace/project/master_img/'

# # Load CSV file containing image file names and labels
# #csv_file = '/home/cygnus/workspace/dl/project/master.csv'  # Update with the path to your CSV file
# csv_file = '/workspace/project/master.csv'  # Update with the path to your CSV file
# df = pd.read_csv(csv_file, header=None, names=['image_file_name', 'label'], skiprows=1)

# # Construct the full file paths for the images
# df['full_image_path'] = df['image_file_name'].apply(lambda x: os.path.join(image_dir, x))

# # Extract full image file paths and labels from the DataFrame
# image_file_paths = df['full_image_path'].tolist()
# labels = df['label'].tolist()

# # Initialize empty lists to store images and labels
# images = []
# label_list = []

# for image_file_path, label in zip(image_file_paths, labels):
#     # Load the image using OpenCV
#     image = cv2.imread(image_file_path)

#     # Check if the image was loaded successfully
#     if image is not None:
#         # Preprocess the image
#         image = image.astype(np.float32) / 255.0  # Normalize pixel values to [0, 1] if necessary

#         # Append the image to the list
#         images.append(image)

#         # Append the label to the list (perform one-hot encoding or label encoding if needed)
#         label_list.append(label)
#     else:
#         print(f"Failed to load image at {image_file_path}")

# # Convert the `images` and `label_list` lists to NumPy arrays
# images_array = np.array(images)
# labels_array = np.array(label_list)

# print(len(images_array))
# print(len(labels_array))

6396
6396


In [2]:
#Shuffle 65b35d1c15f3c6efc1acba1fa1e4c5313541a413edfbcb4a
import random
#1-step in data shuffling
random_seed=47
#get equally spaced numbers in a given range
n = np.arange(images_array.shape[0])
print("'n' values before shuffling = ",n)
print(images_array.shape, labels_array.shape)

#shuffle all the equally spaced values in list 'n'
np.random.seed(random_seed)
np.random.shuffle(n)
print("\n'n' values after shuffling = ",n)
print(images_array.shape, labels_array.shape)

'n' values before shuffling =  [   0    1    2 ... 6393 6394 6395]
(6396, 321, 304, 3) (6396,)

'n' values after shuffling =  [4933 2685 5174 ... 3336 1926 5255]
(6396, 321, 304, 3) (6396,)


In [ ]:
# #2-step in data shuffling
# images_array = images_array[n]
# labels_array = labels_array[n]

# print("Images shape after shuffling = ",images_array.shape,"\nLabels shape after shuffling = ",labels_array.shape)